#### logistic regression has binary op
#### multinomial regression is what you use for multiple categories.. while mathematically sane... DNN's give decent performance without much of the mathematical wrangling

In [1]:
install.packages("mlbench")

package 'mlbench' successfully unpacked and MD5 sums checked

The downloaded binary packages are in
	C:\Users\IBM_ADMIN\AppData\Local\Temp\RtmpIvbsaB\downloaded_packages


In [2]:
data(BreastCancer, package="mlbench")
bc <- BreastCancer[complete.cases(BreastCancer), ]  # create copy

In [4]:
str(bc)

'data.frame':	683 obs. of  11 variables:
 $ Id             : chr  "1000025" "1002945" "1015425" "1016277" ...
 $ Cl.thickness   : Ord.factor w/ 10 levels "1"<"2"<"3"<"4"<..: 5 5 3 6 4 8 1 2 2 4 ...
 $ Cell.size      : Ord.factor w/ 10 levels "1"<"2"<"3"<"4"<..: 1 4 1 8 1 10 1 1 1 2 ...
 $ Cell.shape     : Ord.factor w/ 10 levels "1"<"2"<"3"<"4"<..: 1 4 1 8 1 10 1 2 1 1 ...
 $ Marg.adhesion  : Ord.factor w/ 10 levels "1"<"2"<"3"<"4"<..: 1 5 1 1 3 8 1 1 1 1 ...
 $ Epith.c.size   : Ord.factor w/ 10 levels "1"<"2"<"3"<"4"<..: 2 7 2 3 2 7 2 2 2 2 ...
 $ Bare.nuclei    : Factor w/ 10 levels "1","2","3","4",..: 1 10 2 4 1 10 10 1 1 1 ...
 $ Bl.cromatin    : Factor w/ 10 levels "1","2","3","4",..: 3 3 3 3 3 9 3 3 1 2 ...
 $ Normal.nucleoli: Factor w/ 10 levels "1","2","3","4",..: 1 2 1 7 1 7 1 1 1 1 ...
 $ Mitoses        : Factor w/ 9 levels "1","2","3","4",..: 1 1 1 1 1 1 1 1 5 1 ...
 $ Class          : Factor w/ 2 levels "benign","malignant": 1 1 1 1 1 2 1 1 1 1 ...


All are categorical data/ factors as R calls them

In [5]:
glm(Class ~ Cell.shape, family="binomial", data = bc)


Call:  glm(formula = Class ~ Cell.shape, family = "binomial", data = bc)

Coefficients:
 (Intercept)  Cell.shape.L  Cell.shape.Q  Cell.shape.C  Cell.shape^4  
       4.189        20.911         6.848         5.763        -1.267  
Cell.shape^5  Cell.shape^6  Cell.shape^7  Cell.shape^8  Cell.shape^9  
      -4.439        -5.183        -3.013        -1.289        -0.860  

Degrees of Freedom: 682 Total (i.e. Null);  673 Residual
Null Deviance:	    884.4 
Residual Deviance: 243.6 	AIC: 263.6

glm: generalized liner models 
Class the output is modelled on cell shape alone as its input..  
Cell shape column's values got converted into what looks like binary format  


 Ord.factor cell shape's type: ordered factor (categorical with order as in 1st 2nd 3rd etc...)

In [6]:
# remove id column
bc <- bc[,-1]

# convert factors to numeric
for(i in 1:9) {
 bc[, i] <- as.numeric(as.character(bc[, i]))
}

When converting a factor to a numeric variable  
Always convert it to character and then to numeric,   
else, the values can get screwed up

In [7]:
str(bc)

'data.frame':	683 obs. of  10 variables:
 $ Cl.thickness   : num  5 5 3 6 4 8 1 2 2 4 ...
 $ Cell.size      : num  1 4 1 8 1 10 1 1 1 2 ...
 $ Cell.shape     : num  1 4 1 8 1 10 1 2 1 1 ...
 $ Marg.adhesion  : num  1 5 1 1 3 8 1 1 1 1 ...
 $ Epith.c.size   : num  2 7 2 3 2 7 2 2 2 2 ...
 $ Bare.nuclei    : num  1 10 2 4 1 10 10 1 1 1 ...
 $ Bl.cromatin    : num  3 3 3 3 3 9 3 3 1 2 ...
 $ Normal.nucleoli: num  1 2 1 7 1 7 1 1 1 1 ...
 $ Mitoses        : num  1 1 1 1 1 1 1 1 5 1 ...
 $ Class          : Factor w/ 2 levels "benign","malignant": 1 1 1 1 1 2 1 1 1 1 ...


In [8]:
bc$Class <- ifelse(bc$Class == "malignant", 1, 0)
bc$Class <- factor(bc$Class, levels = c(0, 1))

 encode the response variable into a factor variable of 1's and 0's

## Class imbalance: Deal with it :p

In [9]:
table(bc$Class)


  0   1 
444 239 

Non malignant class seems to be double the number of malignant classes...

#### Here's how to deal with it

    Down sampling, the majority class is randomly down sampled to be of the same size as the smaller class in this case rows with the benign Class will be picked fewer times during the random sampling  
    UpSampling, rows from the minority class, that is, malignant is repeatedly sampled over and over till it reaches the same size as the majority class (benign)  
    ybrid sampling, artificial data points are generated and are systematically added around the minority class.   
    This can be implemented using the SMOTE and ROSE packages in R

In [10]:
install.packages("caret")

also installing the dependencies 'minqa', 'nloptr', 'RcppEigen', 'lme4', 'MatrixModels', 'pbkrtest', 'quantreg', 'car'



package 'minqa' successfully unpacked and MD5 sums checked
package 'nloptr' successfully unpacked and MD5 sums checked
package 'RcppEigen' successfully unpacked and MD5 sums checked
package 'lme4' successfully unpacked and MD5 sums checked
package 'MatrixModels' successfully unpacked and MD5 sums checked
package 'pbkrtest' successfully unpacked and MD5 sums checked
package 'quantreg' successfully unpacked and MD5 sums checked
package 'car' successfully unpacked and MD5 sums checked
package 'caret' successfully unpacked and MD5 sums checked

The downloaded binary packages are in
	C:\Users\IBM_ADMIN\AppData\Local\Temp\RtmpIvbsaB\downloaded_packages


In [11]:
library(caret)
'%ni%' <- Negate('%in%')  # define 'not in' func
options(scipen=999)  # prevents printing scientific notations.

# Prep Training and Test data.
set.seed(100)
trainDataIndex <- createDataPartition(bc$Class, p=0.7, list = F)  # 70% training data
trainData <- bc[trainDataIndex, ]
testData <- bc[-trainDataIndex, ]

Loading required package: lattice
Loading required package: ggplot2


In [12]:
table(trainData$Class)


  0   1 
311 168 

downsample it using the downSample function from caret package.

In [13]:
# Down Sample
set.seed(100)
down_train <- downSample(x = trainData[, colnames(trainData) %ni% "Class"],
                         y = trainData$Class)

table(down_train$Class)


  0   1 
168 168 

Yaaaay !!!!!!!!!! done  
the %in% function is used to select all the columns except the Class column
downSample function requires the 'y' as a factor variable 

In [14]:
# Up Sample.
set.seed(100)
up_train <- upSample(x = trainData[, colnames(trainData) %ni% "Class"],
                     y = trainData$Class)

table(up_train$Class)


  0   1 
311 311 

Build a logistic regression model   
Class as the op variable  dependent variables thickness size shape  
using the downsampled data as input

In [15]:
# Build Logistic Model
logitmod <- glm(Class ~ Cl.thickness + Cell.size + Cell.shape, family = "binomial", data=down_train)

summary(logitmod)


Call:
glm(formula = Class ~ Cl.thickness + Cell.size + Cell.shape, 
    family = "binomial", data = down_train)

Deviance Residuals: 
    Min       1Q   Median       3Q      Max  
-3.0716  -0.1534  -0.0316   0.0193   2.8420  

Coefficients:
             Estimate Std. Error z value        Pr(>|z|)    
(Intercept)   -8.8755     1.3285  -6.681 0.0000000000237 ***
Cl.thickness   0.7293     0.1768   4.125 0.0000370162955 ***
Cell.size      1.0803     0.3334   3.240         0.00119 ** 
Cell.shape     0.8571     0.2920   2.935         0.00334 ** 
---
Signif. codes:  0 '***' 0.001 '**' 0.01 '*' 0.05 '.' 0.1 ' ' 1

(Dispersion parameter for binomial family taken to be 1)

    Null deviance: 465.795  on 335  degrees of freedom
Residual deviance:  70.378  on 332  degrees of freedom
AIC: 78.378

Number of Fisher Scoring iterations: 8


In [17]:
pred <- predict(logitmod, newdata = testData, type = "response")

need to set type='response' in order to compute the prediction probabilities  
.5 is the threshold usually for binary output.. 


In [18]:
y_pred_num <- ifelse(pred > 0.5, 1, 0)
y_pred <- factor(y_pred_num, levels=c(0, 1))
y_act <- testData$Class

In [19]:
mean(y_pred == y_act) 

[1] 0.9411765

#### model building with upsampled data 

In [20]:
logitmod_u <- glm(Class ~ Cl.thickness + Cell.size + Cell.shape, family = "binomial", data=up_train)

summary(logitmod_u)


Call:
glm(formula = Class ~ Cl.thickness + Cell.size + Cell.shape, 
    family = "binomial", data = up_train)

Deviance Residuals: 
    Min       1Q   Median       3Q      Max  
-4.2020  -0.1937  -0.0558   0.0454   2.6289  

Coefficients:
             Estimate Std. Error z value             Pr(>|z|)    
(Intercept)   -7.1921     0.6925 -10.385 < 0.0000000000000002 ***
Cl.thickness   0.5434     0.1087   4.998           0.00000058 ***
Cell.size      0.7307     0.2013   3.630             0.000283 ***
Cell.shape     0.8643     0.1839   4.699           0.00000262 ***
---
Signif. codes:  0 '***' 0.001 '**' 0.01 '*' 0.05 '.' 0.1 ' ' 1

(Dispersion parameter for binomial family taken to be 1)

    Null deviance: 862.28  on 621  degrees of freedom
Residual deviance: 164.51  on 618  degrees of freedom
AIC: 172.51

Number of Fisher Scoring iterations: 8


Higher AIC than down sampled data based model...  
Still lets see :)

In [21]:
pred_u <- predict(logitmod_u, newdata = testData, type = "response")

In [22]:
y_pred_num_u <- ifelse(pred_u > 0.5, 1, 0)
y_pred_u <- factor(y_pred_num_u, levels=c(0, 1))
y_act <- testData$Class

In [23]:
mean(y_pred_u == y_act) 

[1] 0.9411765

Same...?